# Data cleaning:

In [256]:
import html
from bs4 import BeautifulSoup
import pandas as pd
from unidecode import unidecode
data_ACM = pd.read_csv(r'ACM.csv')
data_DBLP = pd.read_csv(r'DBLP2_new.csv')
#data_ACM['venue'] = data_ACM['venue'].to_string()

In [257]:
data_ACM['venue'].str.strip()

0                                    International Conference on Management of Data
1                                    International Conference on Management of Data
2                                    International Conference on Management of Data
3                                    International Conference on Management of Data
4                                    International Conference on Management of Data
                                           ...                                     
2289                                                          Very Large Data Bases
2290    The VLDB Journal &mdash; The International Journal on Very Large Data Bases
2291                                                          Very Large Data Bases
2292                                                              ACM SIGMOD Record
2293                                                          Very Large Data Bases
Name: venue, Length: 2294, dtype: object

In [258]:
data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [259]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))

In [260]:
#data_ACM['venue'] = data_ACM['venue'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))
data_ACM['venue'] = data_ACM['venue'].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

In [261]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: unidecode(str(x)))
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: unidecode(str(x)))

In [262]:
data_ACM['title'] = data_ACM['title'].apply(lambda x: x.upper())
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: x.upper())
#data_ACM['venue'] = data_ACM['venue'].str.upper()

In [263]:
data_DBLP['title'] = data_DBLP['title'].apply(lambda x: x.upper())
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: x.upper())
#data_DBLP['venue'] = data_DBLP['venue'].str.upper()

In [264]:
data_ACM.replace("Very Large Data Bases", "VLDB", inplace=True)
data_ACM.replace("International Conference on Management of Data", "SIGMOD Conference", inplace=True)
data_ACM.replace("ACM SIGMOD Record ", "SIGMOD Record", inplace=True)
data_ACM.replace("ACM Transactions on Database Systems (TODS) ", "ACM Trans. Database Syst. ", inplace=True)
data_ACM.replace("The VLDB Journal — The International Journal on Very Large Data Bases ", "VLDB J.", inplace=True)

In [265]:
data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [266]:
data_ACM.to_csv(r'ACM3.csv', encoding='utf-8', index=False)
data_DBLP.to_csv(r'DBLP3.csv', encoding='utf-8', index=False)

In [267]:
print(data_ACM.head(10))
print(data_DBLP.head(10))

       id  \
0  304586   
1  304587   
2  304589   
3  304590   
4  304582   
5  304583   
6  304584   
7  304585   
8  306112   
9  306115   

                                                                     title  \
0                     THE WASA2 OBJECT-ORIENTED WORKFLOW MANAGEMENT SYSTEM   
1  A USER-CENTERED INTERFACE FOR QUERYING DISTRIBUTED MULTIMEDIA DATABASES   
2             WORLD WIDE DATABASE-INTEGRATING THE WEB, CORBA AND DATABASES   
3                                 XML-BASED INFORMATION MEDIATION WITH MIX   
4                     THE CCUBE CONSTRAINT OBJECT-ORIENTED DATABASE SYSTEM   
5                  THE CORNELL JAGUAR PROJECT: ADDING MOBILITY TO PREDATOR   
6     THE ACTIVE MULTISYNC CONTROLLER OF THE CUBETREE STORAGE ORGANIZATION   
7                                        THE JUNGLE DATABASE SEARCH ENGINE   
8            ADEPT: AN AGENT-BASED APPROACH TO BUSINESS PROCESS MANAGEMENT   
9              A COMPONENTIZED ARCHITECTURE FOR DYNAMIC ELECTRONIC MARKETS  

In [268]:
#data_ACM['venue'] = data_ACM['venue'].to_string()

In [269]:
#for col in data_ACM.columns:
#   data_ACM[col] = data_ACM[col].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

In [270]:
#data_ACM.title.str.lstrip()
#data_ACM.authors.str.lstrip()
#data_ACM.venue.str.lstrip()
#print(data_ACM.loc[2160, 'venue'])
#print(data_ACM.dtypes)

In [271]:
#data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
#data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [272]:
#d2['authors'] = data_ACM.apply(lambda x : x.split(","))
#for i, row in data_ACM.iterrows():
 #   data_ACM.at[i, 'authors'] = data_ACM.at[i, 'authors'].split(",")
#data_ACM['authors'] = data_ACM['authors'].str.split(", ")
#data_ACM['authors'] = data_ACM['authors'].str.split(" ")

In [273]:
#html_data=unicodedata.normalize('NFKD',table.to_html()).encode('ascii','ignore')

# Blocking:

In [274]:
import py_entitymatching as em
em.set_key(data_ACM, 'id')
em.set_key(data_DBLP, 'id')
em.to_csv_metadata(data_ACM, r'ACM3_meta.csv')
em.to_csv_metadata(data_ACM, r'DBLP3_meta.csv')

File already exists at ACM3_meta.csv; Overwriting it
Metadata file already exists at ACM3_meta.metadata. Overwriting it
File already exists at DBLP3_meta.csv; Overwriting it
Metadata file already exists at DBLP3_meta.metadata. Overwriting it


True

In [275]:
print(C1)

       _id  ltable_id  rtable_id  \
0        0     304586     304586   
1        1     304587     304587   
2        2     304589     304589   
3        3     304590     304590   
4        4     304582     304582   
...    ...        ...        ...   
2472  2472     950483     950483   
2473  2473     672977     672977   
2474  2474     672980     672980   
2475  2475     945741     945741   
2476  2476     672979     672979   

                                                                                             ltable_title  \
0                                                    THE WASA2 OBJECT-ORIENTED WORKFLOW MANAGEMENT SYSTEM   
1                                 A USER-CENTERED INTERFACE FOR QUERYING DISTRIBUTED MULTIMEDIA DATABASES   
2                                            WORLD WIDE DATABASE-INTEGRATING THE WEB, CORBA AND DATABASES   
3                                                                XML-BASED INFORMATION MEDIATION WITH MIX   
4                     

In [276]:
C1.to_csv(r'C1.csv', encoding='utf-8', index=False)

In [277]:
dfA =C1[['ltable_id', 'ltable_authors', 'ltable_title', 'ltable_venue', 'ltable_year']].copy()
#['ltable_id', 'ltable_authors', 'ltable_title', 'ltable_venue', 'ltable_year']
dfB = C1[['rtable_id', 'rtable_authors', 'rtable_title', 'rtable_venue', 'rtable_year']].copy()
#['rtable_id', 'rtable_authors', 'rtable_title', 'rtable_venue', 'rtable_year']

In [278]:
x = C1.groupby('ltable_title')

In [279]:
print(x)

In [280]:
stacked = pd.concat([data_ACM, data_DBLP], keys= ['ACM', 'DBLP'], ignore_index=True)
#dfA_title = ACM.groupby('title')
#dfB_title = DBLP.groupby('title')

In [281]:
stacked.to_csv(r'stacked.csv', encoding='utf-8', index=False)

In [282]:
grouped = stacked.groupby('title')

In [283]:
print(len(stacked.groupby('title')))

2791


In [284]:
print(len(stacked))

4847


In [285]:
abc = pd.DataFrame(columns=['idDBLP', 'idACM'])

In [286]:
sum=0
grouped.size()
for key, item in grouped:
    g = grouped.get_group(key)
    if g.size == 10:
        abc = abc.append({'idDBLP' : g.get('id').iloc[1], 'idACM' : g.get('id').iloc[0]}, ignore_index=True)
        #print(g.get('id').iloc[1])
        

In [287]:
lev = pd.DataFrame(columns=['idDBLP', 'idACM'])

In [288]:
#compare = em.string_compare_levenshtein(C, 'title', 'title') 
for key, item in grouped:
    g = grouped.get_group(key)
    s = g.size
    if s > 5:
        compare = em.string_compare_levenshtein(g, 'authors', 'authors')
        if compare >= 0.7:
            if s < 10:
                lev = lev.append({'idDBLP' : g.get('id').iloc[1], 'idACM' : g.get('id').iloc[0]}, ignore_index=True)
            
        

AttributeError: module 'py_entitymatching' has no attribute 'string_compare_levenshtein'

In [ ]:
print(abc.head(10))

In [ ]:
abc2 = abc['idACM'].apply(lambda x : str(x))

In [ ]:
abc2 = abc['idDBLP'].apply(lambda x : str(x))

In [ ]:
abc2.sort_values('idDBLP')

In [ ]:
abc2.to_csv(r'abc2.csv', encoding='utf-8', index=False)

In [ ]:
abc.to_csv(r'abc.csv', encoding='utf-8', index=False)

In [ ]:
perf = pd.read_csv(r'DBLP-ACM_perfectMapping.csv')

In [ ]:
perf['idDBLP'].nunique()

In [ ]:
perf['idACM'].nunique()

In [ ]:
perf['idDBLP'].count()

# Blocking 2. way

In [ ]:
#em.set_key(stacked, 'id')
#em.to_csv_metadata(stacked, r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project/stacked_meta.csv')

In [ ]:
#stack2 = em.read_csv_metadata(r'stacked_meta.csv')

In [ ]:
#ACM = em.read_csv_metadata(r'ACM3_meta.csv', key='id')
#DBLP = em.read_csv_metadata(r'DBLP3_meta.csv', key='id')

In [ ]:
#b1 = em.AttrEquivalenceBlocker() # , 'title', 'authors', 'venue', 'year'

In [ ]:
#C1 = b1.block_tables(ACM, DBLP, 'title', 'title', 
 #    l_output_attrs=['title', 'authors', 'venue', 'year'], 
  #   r_output_attrs=['title', 'authors', 'venue', 'year'], allow_missing=True)

In [ ]:
import recordlinkage

In [369]:
indexer = recordlinkage.Index()
#indexer = Block(left_on=['year', 'venue'],
#                right_on=['year', 'venue'])
indexer.block(['venue', 'year'])
#indexer.full()
candidate_links = indexer.index(data_ACM, data_DBLP)

In [370]:
print(candidate_links)

MultiIndex([(   0,    5),
            (   0,   30),
            (   0,   65),
            (   0,  101),
            (   0,  127),
            (   0,  166),
            (   0,  230),
            (   0,  232),
            (   0,  269),
            (   0,  323),
            ...
            (2290, 1465),
            (2290, 1562),
            (2290, 1609),
            (2290, 1637),
            (2290, 1686),
            (2290, 1798),
            (2290, 1991),
            (2290, 2135),
            (2290, 2338),
            (2290, 2511)],
           length=151972)


In [371]:
"""compare_cl = recordlinkage.Compare()
compare_cl.exact("venue", "venue", label="venue")
compare_cl.string("title", "title", method="levenshtein", label="title")
compare_cl.exact("year", "year", label="year")
compare_cl.string("authors", "authors", method="levenshtein", label="authors")
features = compare_cl.compute(candidate_links, data_ACM, data_DBLP)
print(features)"""

'compare_cl = recordlinkage.Compare()\ncompare_cl.exact("venue", "venue", label="venue")\ncompare_cl.string("title", "title", method="levenshtein", label="title")\ncompare_cl.exact("year", "year", label="year")\ncompare_cl.string("authors", "authors", method="levenshtein", label="authors")\nfeatures = compare_cl.compute(candidate_links, data_ACM, data_DBLP)\nprint(features)'

In [372]:
compare_cl = recordlinkage.Compare()
compare_cl.exact("venue", "venue", label="venue")
compare_cl.string("title", "title", method="jarowinkler", label="title")
compare_cl.exact("year", "year", label="year")
compare_cl.string("authors", "authors", method="jarowinkler", label="authors")
features = compare_cl.compute(candidate_links, data_ACM, data_DBLP)

In [373]:
features

venue     title  year   authors
0    5         1  0.641764     1  0.545622
     30        1  0.675314     1  0.572156
     65        1  0.663559     1  0.376344
     101       1  0.641602     1  0.534008
     127       1  0.543363     1  0.505615
...          ...       ...   ...       ...
2290 1798      1  0.538269     1  0.558967
     1991      1  0.585738     1  0.720678
     2135      1  0.550294     1  0.596184
     2338      1  0.507937     1  0.639254
     2511      1  0.526685     1  0.619514

[151972 rows x 4 columns]

In [374]:
matches = features[features[['title','authors']].sum(axis=1)>= 1.56]

In [375]:
matches.to_csv('matches.csv')

In [376]:
matches.index.names = ['ACM','DBLP']
matches=matches.reset_index()
matches_new=matches.loc[matches.groupby(['ACM'])['DBLP'].idxmax()]
links_pred=matches_new.set_index(['ACM', 'DBLP'])

In [377]:
links_pred.to_csv('predictions.csv')

In [378]:
print(data_ACM.iloc[2])
print(data_DBLP.iloc[440])

id                                                                                                304589
title                                       WORLD WIDE DATABASE-INTEGRATING THE WEB, CORBA AND DATABASES
authors    ATHMAN BOUGUETTAYA, BOUALEM BENATALLAH, LILY HENDRA, JAMES BEARD, KEVIN SMITH, MOURAD QUZZANI
venue                                                                                  SIGMOD Conference
year                                                                                                1999
Name: 2, dtype: object
id                                                                           conf/sigmod/BouguettayaBH99
title                                    WORLD WIDE DATABASE - INTEGRATING THE WEB, CORBA, AND DATABASES
authors    KEVIN SMITH, ATHMAN BOUGUETTAYA, MOURAD OUZZANI, JAMES BEARD, LILY HENDRA, BOUALEM BENATALLAH
venue                                                                                  SIGMOD Conference
year                            

In [379]:
print(data_ACM.iloc[11])
print(data_DBLP.iloc[1258])

id                                                                                         306124
title      DISCOVERING INTERNET MARKETING INTELLIGENCE THROUGH ONLINE ANALYTICAL WEB USAGE MINING
authors                                                      ALEX G. BUCHNER, MAURICE D. MULVENNA
venue                                                                               SIGMOD Record
year                                                                                         1998
Name: 11, dtype: object
id                                        journals/sigmod/Bussler02
title      THE ROLE OF B2B ENGINES IN B2B INTEGRATION ARCHITECTURES
authors                                           CHRISTOPH BUSSLER
venue                                                 SIGMOD Record
year                                                           2002
Name: 1275, dtype: object


In [380]:
perf_match = pd.read_csv('DBLP-ACM_perfectMapping.csv', header=0, encoding="utf-8")

In [381]:
d_1 = data_ACM['id'].to_dict()
d_1_flip = {y: x for x, y in d_1.items()}
d_2 = data_DBLP['id'].to_dict()
d_2_flip = {y: x for x, y in d_2.items()}
perf_match['idACM'] = perf_match['idACM'].map(d_1_flip)
perf_match['idDBLP'] = perf_match['idDBLP'].map(d_2_flip)
perf_match = perf_match[['idACM', 'idDBLP']]
print(perf_match)
links_true = pd.MultiIndex.from_frame(perf_match)

       idACM  idDBLP
0      301.0   143.0
1      306.0  1530.0
2      292.0   310.0
3      289.0  2090.0
4      308.0  1744.0
...      ...     ...
2219   597.0   781.0
2220  1736.0  2004.0
2221   523.0  2260.0
2222     NaN   178.0
2223   925.0  2368.0

[2224 rows x 2 columns]


In [382]:
f_score=recordlinkage.fscore(links_true, links_pred.index)

In [383]:
print(f_score)

0.9449626865671641


# Task 2

In [384]:
features['label'] = 0

In [385]:
for i in links_pred.index.tolist():
    features.loc[i, 'label'] = 1